In [ ]:
# Cancer Type Classification using Deep-Learning
## S.Ravichandran

This document will explain how to use genomic expression data for classifying different cancer/tumor sites/types. This workshop is a follow-up to the NCI-DOE Pilot1 benchmark also called TC1. You can read about the project here, https://github.com/ECP-CANDLE/Benchmarks/tree/master/Pilot1/TC1

For classification, we use a Deep-Learning procedure called 1D-Convolutional Neural Network (CONV1D; https://en.wikipedia.org/wiki/Convolutional_neural_network. 
NCI Genomic Data Commons (GDC; https://gdc.cancer.gov/) is the source of RNASeq expression data. 

First we will start with genomic data preparation and then we will show how to use the data to build CONV1D model that can classify different cancer types. Please note that there are more than ways to extract data from GDC. What I am describing is one possible way. 

This is a continuation of data preparation which can be accessed from here, 
https://github.com/ravichas/ML-TC1

# Part-2: Convolutional Neural Network

## Load some libraries

In [26]:
from __future__ import print_function
import os, sys, gzip, glob, json, time, argparse

import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Input, Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten
from keras import optimizers
from keras.optimizers import SGD, Adam, RMSprop
from keras.models import Sequential, Model, model_from_json, model_from_yaml
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau


In [27]:
# Read features and output files
TC1data3 = pd.read_csv("Data/TC1-data3stypes.tsv", sep="\t", low_memory = False)
outcome = pd.read_csv("Data/TC1-outcome-data3stypes.tsv", sep="\t", low_memory=False, header=None)

In [28]:
# outcome[0].value_counts()
outcome = outcome[0].values

In [29]:
def encode(data):
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [30]:
from keras.utils import to_categorical
outcome = encode(outcome)

Shape of data (BEFORE encode): (150,)
Shape of data (AFTER  encode): (150, 3)



In [31]:
# train/test split
X_train, X_test, Y_train, Y_test = train_test_split(TC1data3, outcome,
                                                    train_size=0.75,
                                                    test_size=0.25,
                                                    random_state=123,
                                                    stratify = outcome)

## Parameters

In [32]:
# parameters
conv=[128, 20, 1, 128, 10, 1]
dense=[200,20]
activation='relu'
batch_size=20

# Number of sites
classes=3
conv=[128, 20, 1, 128, 10, 1]
dense = [200,20]
drop = 0.1
feature_subsample = 0
loss='categorical_crossentropy'
# metrics='accuracy'
out_act='softmax'
pool=[1, 10]
# optimizer='sgd'
shuffle = False
epochs=400

optimizer = optimizers.SGD(lr=0.1)
metrics = ['acc']


In [33]:
x_train_len = X_train.shape[1]

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [34]:
filters = 128
filter_len = 20
stride = 1

pool_list = [1,10]
dense_first = False

K.clear_session()

model = Sequential()
dense_first = True

# model.add  CONV1D
model.add(Conv1D(filters = filters,
                 kernel_size = filter_len,
                 strides = stride,
                 padding='valid',
                 input_shape=(x_train_len, 1)))

In [35]:
# Activation
model.add(Activation('relu'))
# MaxPooling
model.add(MaxPooling1D(pool_size = 1))

filters = 128
filter_len = 10
stride = 1

# Conv1D
model.add(Conv1D(filters=filters,
                 kernel_size=filter_len,
                 strides=stride,
                 padding='valid'))
# Activation
model.add(Activation('relu'))
# MaxPooling
model.add(MaxPooling1D(pool_size = 10))

# Flatten
model.add(Flatten())
# Dense
model.add(Dense(200))
# activation
model.add(Activation('relu'))

#dropout
model.add(Dropout(0.1))

#Dense
model.add(Dense(20))
#Activation
model.add(Activation('relu'))

#dropout
model.add(Dropout(0.1))

model.add(Dense(3))
model.add(Activation(out_act))

model.compile( loss= loss,
              optimizer = optimizer,
              metrics = metrics )
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 60464, 128)        2688      
_________________________________________________________________
activation_1 (Activation)    (None, 60464, 128)        0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 60464, 128)        0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 60455, 128)        163968    
_________________________________________________________________
activation_2 (Activation)    (None, 60455, 128)        0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 6045, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 773760)           

In [ ]:
# save
save = '.'
output_dir = "Output"

output_dir = save
if not os.path.exists(output_dir):
        os.makedirs(output_dir)

model_name = 'tc1'
path = '{}/{}.autosave.model.h5'.format(output_dir, model_name)
checkpointer = ModelCheckpoint(filepath=path,
                               verbose=1,
                               save_weights_only=False,
                               save_best_only=True)

csv_logger = CSVLogger('{}/training.log'.format(output_dir))

# SR: change epsilon to min_delta
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=10,
                              verbose=1, mode='auto',
                              min_delta=0.0001,
                              cooldown=0,
                              min_lr=0)
# batch_size = 20
history = model.fit(X_train, Y_train, batch_size=batch_size,
                    epochs=epochs, verbose=1, validation_data=(X_test, Y_test),
                    callbacks = [checkpointer, csv_logger, reduce_lr])

score = model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

# serialize weights to HDF5
model.save_weights("{}/{}.model.h5".format(output_dir, model_name))
print("Saved model to disk")

# load weights into new model
loaded_model_yaml.load_weights('{}/{}.model.h5'.format(output_dir, model_name))
print("Loaded yaml model from disk")

Train on 112 samples, validate on 38 samples
Epoch 1/400
112/112 [==============================] - 192s 2s/step - loss: 1.6281 - acc: 0.2679 - val_loss: 1.1053 - val_acc: 0.3421

Epoch 00001: val_loss improved from inf to 1.10528, saving model to ./tc1.autosave.model.h5
Epoch 2/400
112/112 [==============================] - 204s 2s/step - loss: 1.1123 - acc: 0.3393 - val_loss: 1.0975 - val_acc: 0.3158

Epoch 00002: val_loss improved from 1.10528 to 1.09748, saving model to ./tc1.autosave.model.h5
Epoch 3/400
112/112 [==============================] - 194s 2s/step - loss: 1.0919 - acc: 0.4018 - val_loss: 1.0758 - val_acc: 0.6579

Epoch 00003: val_loss improved from 1.09748 to 1.07583, saving model to ./tc1.autosave.model.h5
Epoch 4/400
112/112 [==============================] - 184s 2s/step - loss: 1.0564 - acc: 0.4821 - val_loss: 1.0592 - val_acc: 0.3421

Epoch 00004: val_loss improved from 1.07583 to 1.05917, saving model to ./tc1.autosave.model.h5
Epoch 5/400
112/112 [==============